# グラムシュミットMTSのエラー原因を探す

In [46]:
import math
import time

from tqdm import tqdm

import pandas as pd
import numpy as np
import random
from sklearn.utils import resample


from sklearn.utils import resample
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix



# 自作関数
from dataload.data_load import data_load
from func.my_func \
import \
    fit_MTS, predict_MD, determine_threshold, predict_MTSBag_ImpAgg, make_result_df, \
    fit_WMTS, cal_WMD_by_reduced_model, predict_WMTS, predict_WMTSBag_ImpAgg, \
    fit_MTGS, predict_gram_MD, transform_standard, create_gram_vec_U, fit_gram_schmidt, gram_schmidt_cal_MD

In [47]:
print('実験開始')

ex_name = input('実験名は?')

n_experiment = 10
data_list = [
    # 'yeast', 
    'wine', 
    'abalone', 
    'car',
    'cancer', 
    # 'letter'
    ]

print('MTGS開始！')

for data in data_list:
    print(data)
    result_df = pd.DataFrame(
                        columns=['AUC', 'accuracy', 'recall', 'Specificity', 'precision', 'gmeans', 'RS'],
                        index=range(n_experiment))
    for m in tqdm(range(n_experiment)):
        
        X, y = data_load(data)

        # バギング側の話
        # ブートストラップサンプリングの個数
        n = 10
        seed = random.randint(0, n)

        # 使用する7つの変数をランダムに取得する
        # バギングをする際はそれぞれのサブサンプルで7つの変数を選択する．
        random.seed(seed)
        random_s = random.sample(list(X.columns), len(X.columns) if len(X.columns) < 7 else 7)
        X = X[random_s]

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

        result_scaler, result_t, result_ips, select_columns = fit_MTGS(X_train, y_train)

        y_train_pred = predict_gram_MD(X_train, result_scaler, result_t, result_ips, select_columns)

        threshold = determine_threshold(y_train, y_train_pred)

        y_proba = predict_gram_MD(X_test, result_scaler, result_t, result_ips, select_columns)
        y_pred = y_proba > threshold

        result_df = make_result_df(result_df, y_test, y_pred, y_proba, m)
        
    result_df.to_csv(f'../data/output/{ex_name}_MTGS_{data}_result.csv')



実験開始
MTGS開始！
wine


 20%|██        | 2/10 [00:00<00:00, 16.28it/s]

[[ 0.          0.          0.          0.          0.          0.
   0.        ]
 [ 0.11184533  0.          0.          0.          0.          0.
   0.        ]
 [ 0.14675413 -0.16452305  0.          0.          0.          0.
   0.        ]
 [ 0.00988651  0.03372461 -0.03645116  0.          0.          0.
   0.        ]
 [ 0.26345797 -0.63217905 -0.06729526  0.1227749   0.          0.
   0.        ]
 [ 0.08897802  0.10208417  0.08542866 -0.48585672  0.25038477  0.
   0.        ]
 [ 0.10899304  0.30802977 -0.17279136  0.35587014 -0.18027504 -0.61467599
   0.        ]]
ここ 2
[[ 0.          0.          0.          0.          0.          0.
   0.        ]
 [ 0.51927059  0.          0.          0.          0.          0.
   0.        ]
 [ 0.21226369  0.39404758  0.          0.          0.          0.
   0.        ]
 [ 0.30375249  0.29282252  0.04115808  0.          0.          0.
   0.        ]
 [-0.19202385  0.14153208 -0.23567609  0.21042502  0.          0.
   0.        ]
 [ 0.64453332 

 60%|██████    | 6/10 [00:00<00:00, 15.88it/s]

[[ 0.          0.          0.          0.          0.          0.
   0.        ]
 [ 0.45536955  0.          0.          0.          0.          0.
   0.        ]
 [ 0.21433804  0.25650115  0.          0.          0.          0.
   0.        ]
 [ 0.13385649  0.08964038  0.01616581  0.          0.          0.
   0.        ]
 [-0.24434549  0.05437784 -0.32755664  0.12125981  0.          0.
   0.        ]
 [ 0.63836989 -0.13612807  0.29139394  0.30968794 -0.07353385  0.
   0.        ]
 [ 0.3198106   0.11086776 -0.56031747  0.13201584 -0.05698572 -0.23938831
   0.        ]]
ここ 3
[[ 0.          0.          0.          0.          0.          0.
   0.        ]
 [ 0.11347746  0.          0.          0.          0.          0.
   0.        ]
 [ 0.30375156  0.18635567  0.          0.          0.          0.
   0.        ]
 [-0.45841629  0.09393     0.05724079  0.          0.          0.
   0.        ]
 [ 0.0063388   0.04043432  0.49743724  0.22933836  0.          0.
   0.        ]
 [ 0.21935323 

...

 80%|████████  | 8/10 [00:01<00:00,  5.36it/s]

[[ 0.          0.          0.          0.          0.          0.
   0.        ]
 [ 0.71600419  0.          0.          0.          0.          0.
   0.        ]
 [ 0.19807364  0.26779598  0.          0.          0.          0.
   0.        ]
 [ 0.10478253  0.04408523  0.07491594  0.          0.          0.
   0.        ]
 [ 0.25287035  0.23004218  0.13182005 -0.15415934  0.          0.
   0.        ]
 [ 0.36495152  0.50606523  0.25239312 -0.07631713  0.02886481  0.
   0.        ]
 [ 0.30095703  0.21463264  0.10543595 -0.04852333  0.09475902  0.02727717
   0.        ]]
ここ 4
[[ 0.          0.          0.          0.          0.          0.
   0.        ]
 [-0.05299277  0.          0.          0.          0.          0.
   0.        ]
 [-0.18860276  0.36047125  0.          0.          0.          0.
   0.        ]
 [-0.03887847  0.23957085  0.14403569  0.          0.          0.
   0.        ]
 [ 0.1218302   0.07027044  0.23268424  0.31602086  0.          0.
   0.        ]
 [ 0.04176704 

 90%|█████████ | 9/10 [00:01<00:00,  5.32it/s]

 2
[[ 0.          0.          0.          0.          0.          0.
   0.        ]
 [ 0.06102081  0.          0.          0.          0.          0.
   0.        ]
 [ 0.19269245  0.22072542  0.          0.          0.          0.
   0.        ]
 [-0.04684912  0.17958506  0.14504413  0.          0.          0.
   0.        ]
 [-0.05719155  0.13059139 -0.19616236  0.118274    0.          0.
   0.        ]
 [-0.02309728  0.36293178  0.15429987  0.22356991 -0.0267291   0.
   0.        ]
 [-0.02274884  0.33633692  0.14864195  0.12115618 -0.05361379  0.09679368
   0.        ]]
ここ 4


100%|██████████| 10/10 [00:01<00:00,  5.27it/s]


In [16]:
data

'yeast'

In [7]:
X, y = data_load(data)

# バギング側の話
# ブートストラップサンプリングの個数
n = 10
seed = random.randint(0, n)

# 使用する7つの変数をランダムに取得する
# バギングをする際はそれぞれのサブサンプルで7つの変数を選択する．
random.seed(seed)
random_s = random.sample(list(X.columns), len(X.columns) if len(X.columns) < 7 else 7)
X = X[random_s]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)




In [12]:
# 正常データのみを使用して標準化
normal_Z = transform_standard(X[y == 0], X[y == 0])
anomaly_Z = transform_standard(X[y == 0], X[y == 1])

# 正常データのみを使用して t, ips を計算
t, ips = fit_gram_schmidt(normal_Z)
print(t)
anomaly_U = create_gram_vec_U(anomaly_Z, t)

# いったん飛ばす，削除の基準は？削除しない方法もあるっぽい？
    #１度目の仮のマハラノビス距離を計算
    # MD_1st = cal_MD(normal_Z, inv_C)
    # もしもマハラノビス距離が余りにも大きいサンプルがあれば任意で削除する
    # 削除後のデータを使用して標準化と共分散行列を計算

# 異常データと直交表を用いてSN比を計算
#L8直行表
l8 = np.array([
    [1,1,1,1,1,1,1],
    [1,1,1,2,2,2,2],
    [1,2,2,1,1,2,2],
    [1,2,2,2,2,1,1],
    [2,1,2,1,2,1,2],
    [2,1,2,2,1,2,1],
    [2,2,1,1,2,2,1],
    [2,2,1,2,1,1,2]
    ])
l8 = (l8 == 1)

#異常データのマハラノビス距離
result = np.zeros((l8.shape[0], anomaly_Z.shape[0]))
for i, l8_row in enumerate(l8):
    result[i] = gram_schmidt_cal_MD(anomaly_U.T[:, l8_row], ips[l8_row])


[[ 0.          0.          0.          0.          0.          0.
   0.        ]
 [ 0.43882733  0.          0.          0.          0.          0.
   0.        ]
 [-0.08836234 -0.07562096  0.          0.          0.          0.
   0.        ]
 [-0.23296273 -0.01796276 -0.14152947  0.          0.          0.
   0.        ]
 [ 0.00810655  0.11963246 -0.17158563 -0.09806187  0.          0.
   0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.        ]
 [ 0.57435947  0.34242607  0.00570038 -0.0198588  -0.16322951         nan
   0.        ]]


c:\Users\baseb\Documents\github\修論\SMOTEMTSBag\MTSBag\code\func\my_func.py:310: RuntimeWarning: invalid value encountered in double_scalars
  t[l][q] = np.dot(normal_A[l], normal_U[q]) / np.dot(normal_U[q], normal_U[q])
c:\Users\baseb\Documents\github\修論\SMOTEMTSBag\MTSBag\code\func\my_func.py:353: RuntimeWarning: invalid value encountered in double_scalars
  sigma_MD += feature_weight[q] * (u**2 / ips[q])
c:\Users\baseb\Documents\github\修論\SMOTEMTSBag\MTSBag\code\func\my_func.py:353: RuntimeWarning: divide by zero encountered in double_scalars
  sigma_MD += feature_weight[q] * (u**2 / ips[q])


In [19]:
X, _ = data_load(data)
pd.DataFrame(X).describe()

,0,1,2,3,4,5,6,7
count,1484.000000,1484.000000,1484.000000,1484.000000,1484.000000,1484.000000,1484.000000,1484.000000
mean,0.500121,0.499933,0.500034,0.261186,0.504717,0.007500,0.499885,0.276199
std,0.137299,0.123924,0.086670,0.137098,0.048351,0.075683,0.057797,0.106491
min,0.110000,0.130000,0.210000,0.000000,0.500000,0.000000,0.000000,0.000000
25%,0.410000,0.420000,0.460000,0.170000,0.500000,0.000000,0.480000,0.220000
50%,0.490000,0.490000,0.510000,0.220000,0.500000,0.000000,0.510000,0.220000
75%,0.580000,0.570000,0.550000,0.320000,0.500000,0.000000,0.530000,0.300000
max,1.000000,1.000000,1.000000,1.000000,1.000000,0.830000,0.730000,1.000000


In [15]:
pd.DataFrame(X)

,0,3,6,4,7,5,1
0,0.58,0.13,0.48,0.5,0.22,0.0,0.61
1,0.43,0.27,0.53,0.5,0.22,0.0,0.67
2,0.64,0.15,0.53,0.5,0.22,0.0,0.62
3,0.58,0.13,0.54,0.5,0.22,0.0,0.44
4,0.42,0.54,0.48,0.5,0.22,0.0,0.44
...,...,...,...,...,...,...,...
1479,0.81,0.17,0.53,0.5,0.22,0.0,0.62
1480,0.47,0.40,0.48,0.5,0.47,0.0,0.43
1481,0.67,0.19,0.56,0.5,0.22,0.0,0.57
1482,0.43,0.16,0.53,0.5,0.39,0.0,0.40


In [24]:
(X[5]>0).sum()

15

In [29]:
pd.DataFrame(normal_Z)

,0,1,2,3,4,5,6
0,0.871807,-0.342484,0.495790,-0.141421,-0.497913,0.0,0.644796
1,0.871807,0.945383,0.662141,-0.141421,-0.497913,0.0,1.148132
2,0.430964,-0.020517,1.327544,-0.141421,-0.497913,0.0,-0.277986
3,0.116076,1.508825,-1.833120,-0.141421,2.041441,0.0,1.232021
4,-0.198811,-0.101009,0.163089,-0.141421,-0.497913,0.0,0.057571
5,-1.143475,-0.986418,0.163089,-0.141421,-0.497913,0.0,-1.536326
6,0.682874,-1.952319,0.329440,-0.141421,-0.497913,0.0,-0.194097
7,-0.072856,-0.101009,0.163089,-0.141421,-0.497913,0.0,-0.361875
8,-0.954542,-0.986418,0.163089,-0.141421,-0.497913,0.0,-0.110208
9,1.249672,-0.503468,0.495790,-0.141421,-0.497913,0.0,0.057571


In [37]:
t, ips = fit_gram_schmidt(normal_Z)
normal_U = create_gram_vec_U(normal_Z, t)
pd.DataFrame(normal_U.T)

c:\Users\baseb\Documents\github\修論\SMOTEMTSBag\MTSBag\code\func\my_func.py:310: RuntimeWarning: invalid value encountered in double_scalars
  t[l][q] = np.dot(normal_A[l], normal_U[q]) / np.dot(normal_U[q], normal_U[q])


,0,1,2,3,4,5,6
0,0.871807,-0.725057,0.517996,0.121965,-3.173988e-01,0.0,NaN
1,0.871807,0.562810,0.781736,0.182426,-4.202866e-01,0.0,NaN
2,0.430964,-0.209636,1.349772,0.146244,-2.303844e-01,0.0,NaN
3,0.116076,1.457888,-1.712616,-0.330578,1.539812e+00,0.0,NaN
4,-0.198811,-0.013765,0.144481,-0.167536,-4.862922e-01,0.0,NaN
5,-1.143475,-0.484630,0.025401,-0.412919,-4.667986e-01,0.0,NaN
6,0.682874,-2.251983,0.219483,0.008274,-1.955666e-01,0.0,NaN
7,-0.072856,-0.069038,0.151430,-0.138202,-4.766319e-01,0.0,NaN
8,-0.954542,-0.567539,0.035826,-0.368918,-4.523081e-01,0.0,NaN
9,1.249672,-1.051858,0.526672,0.205351,-2.717003e-01,0.0,NaN


In [33]:
1e-100 < 0.000000001

True